# Quantify subgenome phylogenetic signal in UTEX2797
Plot signal as heatmap in circos format.

In [ ]:
from pybedtools import BedTool
from matplotlib import pyplot as plt
import glob

In [ ]:
# parameters
interval_size = 500000
support_threshold = 50
min_band_size = 15

# infiles
faifile = '../../figshare/scaffolded_assemblies/UTEX2797_scaffolds_v1.fasta.fai'
gfffile =  '../../figshare/annotation/genes_scaffolded_assembly/UTEX2797_v1.gff'
synmapfile = '../../figshare/coge/UTEX2797vself/64270_64270.CDS-CDS.last.tdd10.cs0.filtered.dag.all.go_D20_g10_A5.aligncoords.Dm0.ma1.txt'

orange_bedfile = '../../figshare/orthofinder/Comparative_Genomics_Statistics/UTEX2797_sisrels_orange' + str(support_threshold) + '.bed'
pink_bedfile = '../../figshare/orthofinder/Comparative_Genomics_Statistics/UTEX2797_sisrels_pink' + str(support_threshold) + '.bed'

# outfiles
statsfile = '../../figshare/coge/circos/stats.txt'
outdir = '../../figshare/coge/circos/UTEX2797_subgenomes_interval' + str(interval_size) + '_support' + str(support_threshold) + '_minbandsize' + str(min_band_size)
configfile = outdir + '/circos.config'
karyotypefile = outdir + '/karyotype.txt'
linkfile = outdir +'/syntenic_bands.txt'

interval_bedfile = outdir + '/intervals_' + str(interval_size) + '.bed'
genes_bedfile = outdir + '/all_genes.bed'

allgenesint_bedfile = outdir + '/all_genes_intersect_intervals.bed'
orangegenesint_bedfile = outdir + '/orange_genes_intersect_intervals.bed'
pinkgenesint_bedfile = outdir + '/pink_genes_intersect_intervals.bed'

orange_circosfile = outdir + '/orange_subgenome_intervals.txt'
pink_circosfile = outdir + '/pink_subgenome_intervals.txt'

!mkdir -p {outdir}

## Create circos karyotype

In [ ]:
scaffold_length_dict = {}

fi = open(faifile)
fo = open(karyotypefile, 'w')

for line in fi:
    line = line.rstrip()

    scaffold = line.split('\t')[0]
    length = int(line.split('\t')[1])
    
    scaffold_length_dict[scaffold] = length
    fo.write('chr\t-\t' + scaffold + '\t' + scaffold.split('-Scaf')[1] + '\t0\t' + str(length) + '\t' + 'black' + '\n')

fi.close()
fo.close()

## Store interval and gene information as bed files

In [ ]:
fo = open(interval_bedfile, 'w')

for scaffold in scaffold_length_dict:
    length = scaffold_length_dict[scaffold]
    max_intervals = int(length / interval_size) + 1

    intervals = range(0,max_intervals)

    for interval in intervals:
        intervalid = scaffold + '_' + str(interval)
        start = (interval_size * interval) + 1
        stop = interval_size * (interval + 1)

        if stop > length:
            stop = length
        
        fo.write(scaffold + '\t' + str(start) + '\t' + str(stop) + '\t' + intervalid + '\n')

fo.close()

In [ ]:
header = ['genes', 'orange0', 'pink0', 'orange50', 'pink50', 'orange90', 'pink90']

fi = open(gfffile)
fo = open(genes_bedfile, 'w')

counts_dict = {} 

for line in fi:
    feature = line.split('\t')[2]
    
    if feature != 'gene':
        continue

    geneid = line.split('\t')[8].split(';')[0].split('=')[1]
    #print(geneid)

    scaffold = line.split('\t')[0]
    if scaffold not in counts_dict:
        counts_dict[scaffold] = {}
        for geneset in header:
            counts_dict[scaffold][geneset] = []
        
    start = int(line.split('\t')[3])
    stop = int(line.split('\t')[4])

    fo.write(scaffold + '\t' + str(start) + '\t' + str(stop) + '\t' + geneid + '\n')
    counts_dict[scaffold]['genes'].append(geneid)
    
fi.close()
fo.close()

In [ ]:
intervals = BedTool(interval_bedfile)
all_genes = BedTool(genes_bedfile)
orange_genes = BedTool(orange_bedfile)
pink_genes = BedTool(pink_bedfile)

In [ ]:
# Get counts for supplimental table
sisrelfiles = '../../figshare/orthofinder/Comparative_Genomics_Statistics/UTEX2797_sisrels_*.bed'

for infile in glob.glob(sisrelfiles):
    fi = open(infile)
    
    geneset = infile.split('_')[-1].split('.')[0]
    #print(geneset)
    
    for line in fi:
        if line[0] == '#':
            continue
        
        col = line.rstrip().split('\t')
        scaffold = col[0]
                    
        geneid = col[3].split('_')[1]
        counts_dict[scaffold][geneset].append(geneid)
        #print(geneid)
    
    fo.close()

In [ ]:

fo = open(statsfile, 'w')
fo.write('scaffold')

for geneset in header:
    fo.write('\t' + geneset)
fo.write('\n')

for i in range(1,67):
    scaffold = 'UTEX2797-Scaf' + str(i)
    fo.write(scaffold)
    for geneset in header:
        count = len(counts_dict[scaffold][geneset])
        fo.write('\t' + str(count))
    fo.write('\n')

fo.close()

## Calculate intersections between intervals and gene sets

In [ ]:
fo = open(allgenesint_bedfile, 'w')

for interval in intervals.intersect(all_genes, F=0.5, wa=True, wb=True):
    #print(interval)
    fo.write(str(interval))
    
fo.close()

In [ ]:
fo = open(orangegenesint_bedfile, 'w')

for interval in intervals.intersect(orange_genes, F=0.5, wa=True, wb=True):
    #print(interval)
    fo.write(str(interval))
    
fo.close()

In [ ]:
fo = open(pinkgenesint_bedfile, 'w')

for interval in intervals.intersect(pink_genes, F=0.5, wa=True, wb=True):
    #print(interval)
    fo.write(str(interval))
    
fo.close()

In [ ]:
intervalDict = {}

for interval in intervals.intersect(all_genes, c=True, F=0.5):
    intervalDict[interval.name] = [interval.start, interval.stop, interval.score]
    
for interval in intervals.intersect(orange_genes, c=True, F=0.5):
    intervalDict[interval.name].append(interval.score)
    
for interval in intervals.intersect(pink_genes, c=True, F=0.5):
    intervalDict[interval.name].append(interval.score)
    

In [ ]:
orangelist = []
pinklist = []

fo1 = open(orange_circosfile, 'w')
fo2 = open(pink_circosfile, 'w')

for interval in intervalDict:
    orange_count = float(intervalDict[interval][3])
    
    orange_per = 0
    if float(intervalDict[interval][2]) > 0:
        orange_per = orange_count / float(intervalDict[interval][2]) * 100
        
    pink_count = float(intervalDict[interval][4])
    pink_per = 0
    if float(intervalDict[interval][2]) > 0:
        pink_per = pink_count / float(intervalDict[interval][2]) * 100
    
    #print(interval, orange_count, orange_per, pink_count, pink_per)
    scaffold = interval.split('_')[0]
    start = intervalDict[interval][0]
    stop = intervalDict[interval][1]
    
    fo1.write(scaffold + '\t' + str(start) + '\t' + str(stop) + '\t' + str(orange_per) + '\n')
    fo2.write(scaffold + '\t' + str(start) + '\t' + str(stop) + '\t' + str(pink_per) + '\n')
    
    orangelist.append(orange_per)
    pinklist.append(pink_per)
    
fo1.close()
fo2.close()

In [ ]:
print('ORANGE RANGE:', min(orangelist), max(orangelist))
print('PINK RANGE:', min(pinklist), max(pinklist))

## Parse synteny information from quota alignment results downloaded from CoGe

Syntentic blocks identified via DAGChainer 
Input Format: https://genomevolution.org/wiki/index.php/SynMap#Output_Files

In [ ]:
fi = open(synmapfile)
fo = open(linkfile, 'w')

a_scaf = ''
b_scaf = ''
a_start = 1000000000
b_start = 1000000000
a_stop = 0
b_stop = 0
score = 0
num_genes = 0

score_list = []
num_gene_list = []

for line in fi:
    col = line.rstrip().split('\t')
    
    if line.startswith('#'):
        
        if a_scaf != '':
            if num_genes >= min_band_size:
                #print(a_scaf, a_stop, a_stop, b_scaf, b_start, b_stop, score, num_genes)
                fo.write(a_scaf + '\t' + str(a_start) + '\t' + str(a_stop) + '\t' + b_scaf + '\t' + str(b_start) + '\t' + str(b_stop) + '\n')
                score_list.append(score)
                num_gene_list.append(num_genes)
                
            a_scaf = ''
            b_scaf = ''
            a_start = 1000000000
            b_start = 1000000000
            a_stop = 0
            b_stop = 0
            score = 0
            num_genes = 0

        #score = float(col[1])
        #num_genes = int(col[5])
        #print(score, num_genes)
    
    else:
        
        a_scaf = col[1].split('||')[0]
        b_scaf = col[5].split('||')[0]
        
        if a_start > int(col[1].split('||')[1]):
            a_start = int(col[1].split('||')[1])

        if b_start > int(col[5].split('||')[1]):
            b_start = int(col[5].split('||')[1])

        if a_stop < int(col[1].split('||')[2]):
            a_stop = int(col[1].split('||')[2])

        if b_stop < int(col[5].split('||')[2]):
            b_stop = int(col[5].split('||')[2])

        num_genes += 1 

        score = int(col[9])

fi.close()            
fo.close()

Plot distribution of scores for syntenic pairs

In [ ]:
print(min(score_list), '-', max(score_list))
plt.hist(score_list, 50)
plt.show()

plot distribution of gene counts for syntenic pairs

In [ ]:
print(min(num_gene_list), '-', max(num_gene_list))
plt.hist(num_gene_list, 50)
plt.show()

## Prepare circos configuration file

To run:
```circos -conf circos.conf -outputfile sub_genome_circos.png```

In [ ]:
fo = open(configfile, 'w')

fo.write('''
<<include etc/colors_fonts_patterns.conf>>
<<include etc/housekeeping.conf>>
<<include colors_fonts_patterns.conf>>

karyotype = karyotype.txt

show_ticks          = no
show_tick_labels    = no

chromosomes_units           = 1000000
chromosomes_display_default = yes

<image>
	<<include etc/image.conf>>
</image>


<ideogram>
	<spacing>
		default = 0.007r
		break   = 0.5r

		<pairwise UTEX2797-Scaf66 UTEX2797-Scaf1>
			spacing = 5r 
		</pairwise>

	</spacing>

	show_bands       = no
	radius           = 0.90r
	thickness        = 25p
	fill             = yes
	fill_color       = black
	stroke_thickness = 2
	stroke_color     = black

	show_label       = yes
	label_font       = default
	#label_radius    = 0.95r
	label_with_tag   = yes
	label_size       = 40
	label_parallel   = yes
	label_case       = lower
	label_radius     = dims(ideogram,radius_outer) + 25p

</ideogram>


<links>
	<link>
		file          = syntenic_bands.txt
		radius        = 0.81r
		bezier_radius = 0r
		color         = grey
		thickness     = 2
		ribbon           = yes
		flat             = yes
		stroke_color     = grey
		stroke_thickness = 3

		<rules>
			<rule>
				condition = var(intrachr)
				show = no
			</rule>
		</rules>
	</link>
</links>

<plots>

	# orange heatmap
	<plot>
		type  = heatmap
		file  = orange_subgenome_intervals.txt
		r1    = 0.99r
		r0    = 0.90r
		color = ylorbr-9-seq
	</plot>

	# pink heatmap
	<plot>
		type  = heatmap
		file  = pink_subgenome_intervals.txt
		r1    = 0.90r
		r0    = 0.81r
		color = purd-9-seq
	</plot>

</plots>
''')

fo.close()

In [ ]:
### To run Circos:
# module load circos
# cd {outdir}
# circos -conf circos.config -outputfile {outdir}_circos.png